In [1]:
import xmltodict as xd
import pandas as pd
import os, glob
import mio
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon
import pathlib
import time

In [2]:
def parse_model(dae_file:str) -> dict:
    " get point and vertex list for one model"
    with open(dae_file) as fin:
        s = fin.read()
    d = xd.parse(s)
    
    geometries = d['COLLADA']['library_geometries']['geometry']
    
    res = {}
    for geo in geometries:
        geo_id = geo['@id']

        # float list
        f_list = geo['mesh']['source']['float_array']['#text'].split()
        f_list = [float(f) for f in f_list]

        # vertex list
        v_list = geo['mesh']['triangles']['p'].split()
        v_list = [int(n) for n in v_list]
        # write result
        
        res[geo_id] = (f_list, v_list)
    
    return res

In [3]:
def comb_f_v(f_list, v_list):
    va = np.array(v_list)
    va = va.reshape(len(v_list)//3, 3)
    v = pd.DataFrame(va, columns=['v0', 'v1', 'v2'])
    
    fa = np.array(f_list)
    fa = fa.reshape(len(f_list)//3, 3)
    f = pd.DataFrame(fa, columns=['x', 'y', 'z'])
    
    assert max(v.v0.max(), v.v1.max(), v.v2.max()) +1 == len(f)
    
    # combine the lists
    res = (
        v.join(f, on='v0').rename(columns={'x':'x0', 'y':'y0', 'z':'z0'})
         .join(f, on='v1').rename(columns={'x':'x1', 'y':'y1', 'z':'z1'})
         .join(f, on='v2').rename(columns={'x':'x2', 'y':'y2', 'z':'z2'})
         .drop(columns=['v0', 'v1', 'v2'])
    )
    
    return res

In [4]:
def dae2df(dae_path):
    dae_path = pathlib.Path(dae_path)
    dic = parse_model(dae_path)
    ldf = []
    for key in dic:
        f_list, v_list = dic[key]
        df = comb_f_v(f_list, v_list)
        ldf.append(df)
    
    res = pd.concat(ldf)
    res.insert(0, 'model', dae_path.name)
    return res

In [5]:
def get_model_list(kml_path:str)->pd.DataFrame:
    " read kml file, get path and anchor coordinates for every model"
    with open(kml_path) as fin:
        s = fin.read()
    kmld =  xd.parse(s)
    rows = []
    for pm in kmld['kml']['Document']['Placemark'][0:]:
        model = pm['Model']
        row = {}
        row['model'] = os.path.basename(model['Link']['href'])
        sLon = model['Location']['longitude']
        sLat = model['Location']['latitude']
        row['swiss_lon'] = float(sLon)
        row['swiss_lat'] = float(sLat)
        row['swiss_x'], row['swiss_y'] = mio.wgs_swiss(sLon, sLat)
        rows.append(row)
    df = pd.DataFrame(rows)
    
    mdir = str(pathlib.Path(kml_path).parent / 'models')
    res = (
        df.assign(model_path = lambda d : mdir + '\\' + d.model)
        .set_index('model')
    )
    return res

In [6]:
# create datafrme with all triangles
#
def make_tri_tab(kml):
    p_kml = pathlib.Path(kml)
    mod_list = get_model_list(p_kml)
    ldf = []
    i = 0
    for ind, row in mod_list.iterrows():
        i += 1
        mio.show_perc(i, len(mod_list), 100)
        try:
            tris = dae2df(row.model_path)
            tris.insert(0, 'kml', p_kml.name)
            tris.insert(2, 'swiss_x', row.swiss_x)
            tris.insert(3, 'swiss_y', row.swiss_y)
            ldf.append(tris)
        except:
            print(f'Problem with {row.model_path}')
    df = pd.concat(ldf)

    # create a geo dataframe
    vec = (
        pd.DataFrame()
        .assign(kml = df.kml)
        .assign(model = df.model)
        .assign(x0 = df.x0 + df.swiss_x, y0 = df.y0 + df.swiss_y, z0 = df.z0)
        .assign(x1 = df.x1 + df.swiss_x, y1 = df.y1 + df.swiss_y, z1 = df.z1)
        .assign(x2 = df.x2 + df.swiss_x, y2 = df.y2 + df.swiss_y, z2 = df.z2)
    )
    geom = [Polygon( [(t[0], t[1]), (t[2], t[3]), (t[4], t[5])]) for t in zip(vec.x0, vec.y0, vec.x1, vec.y1, vec.x2, vec.y2)]
    gdf = gpd.GeoDataFrame(vec, geometry=geom)
    # filter out vertical walls
    res = gdf[gdf.geometry.area > 0]
    return gdf

In [8]:
def get_tag_todo():
    source_dir = pathlib.Path("C:/Buildings2.0/KML")
    tags_all = set([d.stem for d in source_dir.glob('1091*')])

    dest_dir = pathlib.Path('tab')
    tags_done =  set([t.stem for t in dest_dir.glob('*.tab')])

    tags_todo = tags_all.difference(tags_done)
    l = list(tags_todo)
    l.sort()
    
    if len(l) > 0:
        print(f"{len(l)} files to do")
        return l[0]
    else:
        return

In [ ]:
# MAIN
while(True):
    tag = get_tag_todo()
    if tag is not None:
        print(f'processing {tag}')
        kml = f"C:/Buildings2.0/KML/{tag}/{tag}.kml"
        gdf = make_tri_tab(kml)
        print(f'writing {tag}.tab')
        mio.write_tab(gdf, f'tab/{tag}.tab')
    time.sleep(10)

1 files to do
processing 1091-41
0.76% 1.52% 2.27% 3.03% 3.79% 4.55% 5.31% 6.07% 6.82% 7.58% 8.34% 9.1% 9.86% 10.62% 11.37% 12.13% 12.89% 13.65% Problem with C:\Buildings2.0\KML\1091-41\models\model_1893.dae
14.41% 15.17% 15.92% 16.68% 17.44% 18.2% 18.96% 19.71% 20.47% 21.23% 21.99% 22.75% 23.51% 24.26% 25.02% 25.78% 26.54% 27.3% 28.06% 28.81% 29.57% 30.33% 31.09% 31.85% 32.61% 33.36% 34.12% 34.88% 35.64% 36.4% 37.15% 37.91% 38.67% 39.43% 40.19% 40.95% 41.7% 42.46% 43.22% 43.98% 44.74% 45.5% 46.25% 47.01% 47.77% 48.53% 49.29% 50.05% 50.8% 51.56% 52.32% 53.08% 53.84% 54.6% 55.35% 56.11% 56.87% 57.63% 58.39% 59.14% 59.9% 60.66% 61.42% 62.18% 62.94% Problem with C:\Buildings2.0\KML\1091-41\models\model_8366.dae
63.69% 64.45% 65.21% 65.97% 66.73% 67.49% 68.24% 69.0% 69.76% 70.52% 71.28% 72.04% 72.79% 73.55% 74.31% 75.07% 75.83% 76.58% 77.34% 78.1% 78.86% 79.62% 80.38% 81.13% 81.89% 82.65% 83.41% 84.17% 84.93% 85.68% 86.44% 87.2% 87.96% 88.72% 89.48% 90.23% 90.99% 91.75% 92.51% 93.27% 94.02